# Estimating Default and Asset Correlation with Method of Moments for Latin American Companies from 1997-2020 

This Python notebook reproduces the numerical results of Chapter 6, "Modelling and Estimating Default Correlation with the Asset Value Approach," in "Credit Risk Modeling using Excel and VBA" by Gunter Loffler and Peter Posch.

https://www.wiley.com/en-gb/Credit+Risk+Modeling+using+Excel+and+VBA%2C+2nd+Edition-p-9780470660928.

Remarkably, the authors carried out the original work in Excel. However, the person who wrote this script, Domenico Picone, implemented the same algorithms in Python. This is the modified version of that script, which computes default and asset correlations for the Latin American companies from 1997 to 2020.

There are two main assuptions: 1) All obligors share the same defaut and joint default probabilities as the other obligors in the same Specuative Grade credit group. 2) There is no serial correlation in the time series of defaults, so defaults in any year are not affected by the defaults in a previous year.

In [63]:
import pandas as pd
import numpy  as np

from scipy.stats     import norm
from scipy.optimize  import root
from scipy.integrate import quad
from scipy.optimize  import minimize
from scipy.special   import comb
from scipy.stats     import norm

import matplotlib.pyplot as plt
%matplotlib inline

Let's read our data set, from the book:

In [64]:
DefaultData = pd.read_csv ('DefaultHistoryData.csv', index_col=False)
print (DefaultData.head(2))
print (DefaultData.tail(2))
size = len(DefaultData)

   Year  SpeculativeGradeDefaults  SpeculativeGrade_No
0  1997                         0                   67
1  1998                         0                  126
    Year  SpeculativeGradeDefaults  SpeculativeGrade_No
22  2019                         4                  499
23  2020                        13                  503


In [65]:
print (DefaultData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Year                      24 non-null     int64
 1   SpeculativeGradeDefaults  24 non-null     int64
 2   SpeculativeGrade_No       24 non-null     int64
dtypes: int64(3)
memory usage: 708.0 bytes
None


### Methods of Moments

#### Default Rate

We estimate the average default rate for the Speculative credit categories, $ p_{ig}$ and $p_{spec}$ by averaging their respective annual default rates

In [66]:
DefaultData['DefaultRateSpec']   = DefaultData.SpeculativeGradeDefaults/DefaultData.SpeculativeGrade_No

In [67]:
AverageDefaultRateSpec = np.mean(DefaultData['DefaultRateSpec'])

In [68]:
print(AverageDefaultRateSpec)

0.02499762141064667


#### Joint default rate

We now estimate the average joint default rates as we previously did for the default rates. 

We relate the number of observed joint (pair) defaults to the total number of possible joint (pair) defaults as follows.

Let's indicate with $D_t$ the number of defaults in year $t$ and with $N_t$ the number of exposures at the beginning of year $t$

The number of observed joint (pair) defaults is
$$ {{D_t}\choose 2} =  \frac{D_t(D_t -1)}{2} $$
$$ $$
The total number of possible joint (pair) defaults is
$$ {{N_t}\choose 2} =  \frac{N_t(N_t -1)}{2} $$
$$ $$
The joint default rate for year $t$ is
$$ p_{joint, t}=  \frac{D_t(D_t -1)}{N_t(N_t -1)} $$
$$ $$
and the average joint default rate for all $t's$ is 
$$p_{joint} = \frac{1}{T}\sum_{t=1}^T p_{joint, t}=\frac{1}{T}\sum_{t=1}^T \frac{D_t(D_t -1)}{N_t(N_t -1)}$$

In [69]:
DefaultData['Joint_Def_ProbSpec']   = DefaultData.SpeculativeGradeDefaults *(DefaultData.SpeculativeGradeDefaults - 1) \
/(DefaultData.SpeculativeGrade_No*(DefaultData.SpeculativeGrade_No-1))

In [70]:
JointDefProbSpec = np.mean(DefaultData['Joint_Def_ProbSpec'])

print(np.round(JointDefProbSpec,6))

0.002537


#### Default correlation 

The default correlation is specified by the individual and joint default probabilities. So using the average default and joint default probabilities, we calculate the default correlation as

$$\rho_{i,j}= \frac{p_{i,j}-p_ip_j}{\sqrt{p_i(1-p_j)p_j(1-p_i)}} $$

Again, as stated earlier, we assume all obligors in the same category, share the same default and joint default probability 

In [71]:
DefaultCorrSpec = (JointDefProbSpec - AverageDefaultRateSpec*AverageDefaultRateSpec)\
/(np.sqrt(AverageDefaultRateSpec*(1-AverageDefaultRateSpec)*AverageDefaultRateSpec*(1-AverageDefaultRateSpec)))

#### The asset value model

Instead of imposing the default correlation structure "directly" from the historic default data, more conveniently defaults can represented as function of continuous "random" variables and then a joint default structure is imposed on these variables. 

These variables are often referred as latent variables and are interpreted as the firm's asset value. When the asset value $A_i$ drops below a critical value $d_i$ (called default threshod), default is triggered.

The default indicator for the obligor $i$ can be represented as
$$Default_i = 1 \, ,if \, A_i <= d_i$$

whereas the no default indicator is 

$$No \, default_i = 0 \, ,if \, A_i > d_i$$

The joint default probability is instead written as 
$$Prob(A_i <= d_i \, , A_j <= d_j)$$

Continuing, the asset value $A_i$ depends on the common factor $Z$, which is common to all obligors, on the idiosyncratic factor $\epsilon_i$ and on the factor sensitivity $w_i$

$$A_{i} = w_iZ+\sqrt{1-w_i^2}\epsilon_i$$

The asset correlation between $i$ and $j$ is completely determined by their factor sensitivities $w_i$ and $w_j$ and simplifies to

$$\rho_{i,j}^{asset}= w_iw_j$$

The default probability is given by
$$P[A_i<=d_i] = p_i = \Phi(d_i)$$

and the joint default probability is given by
$$P[A_i<=d_i,A_j<=d_j ] = p_{i,j} = \Phi_{2}(d_i, d_j,\rho_{i,j}^{asset})$$

where 
$\Phi(d_i)$ is the cumulative standard normal distribution function and
$$ $$
$\Phi_{2}(d_i, d_j,\rho_{i,j}^{asset})$ is the bivariate standard normal distribution function with asset correlation $\rho_{i,j}^{asset}$ 


Once a credit is allocated to Speculative Grade credit category, it will share the same default rate, joint defaut rate, critical value (or default threshold), default and asset correlation of all the other credits in the same category. It is a big assumption indeed!

With those assumptions in mind it is easy to find the asset correlation, $ \rho_{i,j}^{asset}$, that allows us to match the joint default rate as calculated earlier.

We introduce the following three functions to parametrize the asset model

In [72]:
def tetrachoric(x , y , rho): 

    FACCURACY = 0.0000000000001
    MinStopK  = 20    
    hx  = 1
    hy  = 1
    hx1 = 0
    hy1 = 0
    k   = 0
    c   = rho
    z   = c
    s   = z
    CheckPass = 0

    while (CheckPass < MinStopK):
        k   = k + 1
        hx2 = hx1
        hy2 = hy1
        hx1 = hx
        hy1 = hy
        hx  = x * hx1 - (k - 1) * hx2
        hy  = y * hy1 - (k - 1) * hy2
        c   = c * rho / (k + 1)
        z   = hx * hy * c
        s   = s + z
        if (abs(z / s) < FACCURACY):
            CheckPass = CheckPass + 1
        else:
            CheckPass = 0
        
    return s

def bivnor(x , y , rho):
    biv = 0.0
    if (rho == 0):
        biv = norm.cdf(x) * norm.cdf(y)
    else:
        biv = norm.cdf(x) * norm.cdf(y) + norm.pdf(x) * norm.pdf(y) * tetrachoric(x, y, rho)
    return biv

def wrapperRoot(x, y, jointDefault):
    def f(rho):
        return bivnor(x,y,rho) - jointDefault
    return f

#### Default Triggers

are below

In [73]:
x_inverseDRSpec = norm.ppf(AverageDefaultRateSpec, loc=0, scale=1)
print(x_inverseDRSpec)

-1.960004684024761


In [74]:
# Let's find now the asset correlation which matches the joint default rate for the Spec credit group
print("Joint Default rate for the Spec credit is", JointDefProbSpec)
ff  = wrapperRoot(x_inverseDRSpec,x_inverseDRSpec,JointDefProbSpec)
sol = root(ff,0.05)
AssetCorrSpec = sol.x[0]
print("Asset Correlation  for the Spec credit is", AssetCorrSpec)

Joint Default rate for the Spec credit is 0.0025365262886004815
Asset Correlation  for the Spec credit is 0.31869546895066586


In [75]:
# so assuming an asset correlation of 0.319, the joint default probability is the same as above
print(bivnor(x_inverseDRSpec,x_inverseDRSpec,0.31869546895066586))
print(JointDefProbSpec)

0.0025365262886004815
0.0025365262886004815


Let' print the moments so far calcuated, which we remind were estimated directly on the historic default data

In [76]:
print("{0:16} {1:18} {2:24} {3:8}".format("Credit Type", "Default Prob", "Joint Default Prob", "Default Correlation"))

print('{0} {1:13.4f} {2:21.8f} {3:26.5}'.format ("Speculative",AverageDefaultRateSpec, JointDefProbSpec, DefaultCorrSpec))

Credit Type      Default Prob       Joint Default Prob       Default Correlation
Speculative        0.0250            0.00253653                   0.078434


Whereas when using an asset model to describe a default, the moments are below

In [77]:
print("{0:16} {1:20} {2:22} {3:8}".format("Credit Type", "Default Threshold", "Joint Default Prob", "Asset Correlation"))

print('{0} {1:13.4f} {2:26.8f} {3:21.5}'.format ("Speculative",x_inverseDRSpec, JointDefProbSpec, AssetCorrSpec))

Credit Type      Default Threshold    Joint Default Prob     Asset Correlation
Speculative       -1.9600                 0.00253653                0.3187
